In [1]:
library(readr)
library(survival)
library(dplyr)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
TENURES <- read_csv('bytenure.csv', na=character()) %>% filter(ending != 'interim only')

Parsed with column specification:
cols(
  slug = col_character(),
  coach = col_character(),
  coach_id = col_character(),
  ending = col_character(),
  total_games = col_double(),
  total_wins = col_double(),
  total_losses = col_double(),
  start_rating = col_double(),
  end_rating = col_double(),
  top = col_double(),
  bottom = col_double(),
  middle = col_double(),
  last_improvement = col_double()
)


In [3]:
head(TENURES)

slug,coach,coach_id,ending,total_games,total_wins,total_losses,start_rating,end_rating,top,bottom,middle,last_improvement
krugelo99c|00|02|ATL,Lon Kruger,krugelo99c,fired,191,69,122,1354.050,1424.825,1498.447,1323.606,1378.746,-12.1864
stottte99c|02|04|ATL,Terry Stotts,stottte99c,fired,137,52,85,1431.213,1404.525,1454.663,1351.423,1405.307,-45.4351
woodsmi01c|04|10|ATL,Mike Woodson,woodsmi01c,fired,493,206,287,1416.497,1625.674,1625.674,1201.652,1411.411,113.4293
drewla01c|10|13|ATL,Larry Drew,drewla01c,fired,230,128,102,1577.661,1481.606,1608.431,1434.701,1535.035,-51.1603
budenmi99c|13|17|ATL,Mike Budenholzer,budenmi99c,,356,195,161,1479.509,1372.671,1701.183,1372.671,1520.065,-122.9691
pitinri99c|00|01|BOS,Rick Pitino,pitinri99c,resigned,34,12,22,1509.496,1372.376,1512.213,1372.376,1459.238,-137.1198


In [4]:
TENURES %>%
group_by(ending) %>%
summarize(ending_freq = n()) %>%
arrange(desc(ending_freq))

ending,ending_freq
fired,114
,30
resigned,26
contract expired,2
mutual,2
retired,2
traded,2
died,1


In [5]:
death_event <- c('fired', 'contract expired', 'mutual')

In [6]:
TENURES <- TENURES %>% mutate(death = (ending %in% death_event)*1)

In [7]:
TENURES %>% head

slug,coach,coach_id,ending,total_games,total_wins,total_losses,start_rating,end_rating,top,bottom,middle,last_improvement,death
krugelo99c|00|02|ATL,Lon Kruger,krugelo99c,fired,191,69,122,1354.050,1424.825,1498.447,1323.606,1378.746,-12.1864,1
stottte99c|02|04|ATL,Terry Stotts,stottte99c,fired,137,52,85,1431.213,1404.525,1454.663,1351.423,1405.307,-45.4351,1
woodsmi01c|04|10|ATL,Mike Woodson,woodsmi01c,fired,493,206,287,1416.497,1625.674,1625.674,1201.652,1411.411,113.4293,1
drewla01c|10|13|ATL,Larry Drew,drewla01c,fired,230,128,102,1577.661,1481.606,1608.431,1434.701,1535.035,-51.1603,1
budenmi99c|13|17|ATL,Mike Budenholzer,budenmi99c,,356,195,161,1479.509,1372.671,1701.183,1372.671,1520.065,-122.9691,0
pitinri99c|00|01|BOS,Rick Pitino,pitinri99c,resigned,34,12,22,1509.496,1372.376,1512.213,1372.376,1459.238,-137.1198,0


In [8]:
POC <- read_csv('../categorized/coach-list-poc.csv')

Parsed with column specification:
cols(
  coach = col_character(),
  coach_id = col_character(),
  franchises = col_character(),
  poc = col_character(),
  note = col_character()
)


In [9]:
head(POC)

coach,coach_id,franchises,poc,note
Tyronn Lue,luety01c,CLE,yes,NA
Tyrone Corbin,corbity01c,SAC|UTA,yes,NA
Tony Brown,brownto01c,NJNBRK,yes,NA
Terry Porter,portete01c,PHO|MIL,yes,NA
Sidney Lowe,lowesi01c,VANMEM,yes,NA
Sam Vincent,vincesa01c,CHARBH,yes,NA


In [10]:
PLAYERS <- read_csv('../categorized/coach-list-qualifications.csv')

Parsed with column specification:
cols(
  coach = col_character(),
  coach_id = col_character(),
  franchises = col_character(),
  former_nba_player = col_character()
)


In [11]:
head(PLAYERS)

coach,coach_id,franchises,former_nba_player
Isiah Thomas,thomais01c,IND|NYK,yes
Michael Cooper,coopemi01c,DEN,yes
Larry Drew,drewla01c,ATL|MIL,yes
Vinny Del Negro,delnevi01c,CHI|LAC,yes
Paul Westphal,westppa01c,SAC|SEAOKC,yes
Mike Woodson,woodsmi01c,NYK|ATL,yes


In [12]:
tenures <- list(
    TENURES,
    POC %>% select(coach_id, poc),
    PLAYERS %>% select(coach_id, former_nba_player)
) %>%
Reduce(function(dtf1,dtf2) left_join(dtf1,dtf2,by="coach_id"), .)

In [13]:
head(tenures)

slug,coach,coach_id,ending,total_games,total_wins,total_losses,start_rating,end_rating,top,bottom,middle,last_improvement,death,poc,former_nba_player
krugelo99c|00|02|ATL,Lon Kruger,krugelo99c,fired,191,69,122,1354.050,1424.825,1498.447,1323.606,1378.746,-12.1864,1,no,no
stottte99c|02|04|ATL,Terry Stotts,stottte99c,fired,137,52,85,1431.213,1404.525,1454.663,1351.423,1405.307,-45.4351,1,no,no
woodsmi01c|04|10|ATL,Mike Woodson,woodsmi01c,fired,493,206,287,1416.497,1625.674,1625.674,1201.652,1411.411,113.4293,1,yes,yes
drewla01c|10|13|ATL,Larry Drew,drewla01c,fired,230,128,102,1577.661,1481.606,1608.431,1434.701,1535.035,-51.1603,1,yes,yes
budenmi99c|13|17|ATL,Mike Budenholzer,budenmi99c,,356,195,161,1479.509,1372.671,1701.183,1372.671,1520.065,-122.9691,0,no,no
pitinri99c|00|01|BOS,Rick Pitino,pitinri99c,resigned,34,12,22,1509.496,1372.376,1512.213,1372.376,1459.238,-137.1198,0,no,no


In [14]:
regdata <- tenures[
    ,
    !(colnames(tenures) %in% c("slug","coach","coach_id","ending","total_losses","improvement"))
]

In [15]:
surv.all <- coxph(
    Surv(total_games, death) ~ .,
    data = regdata
)

In [16]:
summary(surv.all)

Call:
coxph(formula = Surv(total_games, death) ~ ., data = regdata)

  n= 179, number of events= 118 

                           coef  exp(coef)   se(coef)       z Pr(>|z|)    
total_wins           -0.0835351  0.9198588  0.0074665 -11.188  < 2e-16 ***
start_rating         -0.0049149  0.9950972  0.0020645  -2.381  0.01728 *  
end_rating           -0.0008164  0.9991839  0.0028802  -0.283  0.77683    
top                  -0.0047038  0.9953072  0.0043269  -1.087  0.27698    
bottom                0.0158767  1.0160034  0.0037124   4.277  1.9e-05 ***
middle                0.0134880  1.0135794  0.0051221   2.633  0.00846 ** 
last_improvement     -0.0043636  0.9956459  0.0028089  -1.553  0.12031    
pocyes                0.2925532  1.3398440  0.2144580   1.364  0.17252    
former_nba_playeryes -0.0606292  0.9411722  0.2231530  -0.272  0.78586    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

                     exp(coef) exp(-coef) lower .95 upper .95
total_wins       